In [13]:
import pandas as pd
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import objective_functions
from pypfopt import BlackLittermanModel

# Read in price data
df = pd.read_csv("stock_prices.csv", parse_dates=True, index_col="date")

# Calculate expected returns and sample covariance
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

# Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S) #Add constraints: ef = EfficientFrontier(mu, S, weight_bounds=(0, 0.1)) #Max 10% on each weight.
ef.add_objective(objective_functions.L2_reg, gamma=1) #Reduce zero-weights by adding penalty on small weights with a L2-regularization term
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.save_weights_to_file("weights.csv")  # saves to file
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

OrderedDict({'GOOG': 0.08369, 'AAPL': 0.09537, 'FB': 0.11076, 'BABA': 0.067, 'AMZN': 0.10739, 'GE': 0.02716, 'AMD': 0.0, 'WMT': 0.03077, 'BAC': 0.0, 'GM': 0.0, 'T': 0.02196, 'UAA': 0.01522, 'SHLD': 0.0, 'XOM': 0.0432, 'RRC': 0.0, 'BBY': 0.06553, 'MA': 0.1515, 'PFE': 0.08498, 'JPM': 0.0251, 'SBUX': 0.07038})
Expected annual return: 26.5%
Annual volatility: 21.7%
Sharpe Ratio: 1.13


/home/codespace/.python/current/lib/python3.12/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(


(0.2650387590869301, 0.21662532560561654, 1.1311639504839912)

In [11]:
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices


latest_prices = get_latest_prices(df)

da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=10000)
allocation, leftover = da.greedy_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))

Discrete allocation: {'MA': 20, 'FB': 12, 'PFE': 54, 'BABA': 4, 'AAPL': 4, 'GOOG': 1, 'BBY': 2, 'SBUX': 1}
Funds remaining: $11.89


### Black-Litterman allocation

In [14]:
S = risk_models.sample_cov(df)
viewdict = {"AAPL": 0.20, "BBY": -0.30, "BAC": 0, "SBUX": -0.2, "T": 0.131321}
bl = BlackLittermanModel(S, pi="equal", absolute_views=viewdict, omega="default")
rets = bl.bl_returns()

ef = EfficientFrontier(rets, S)
ef.max_sharpe()

OrderedDict([('GOOG', 0.0),
             ('AAPL', 0.2516700243034513),
             ('FB', 0.0061410740896274),
             ('BABA', 0.0504178900290811),
             ('AMZN', 0.0),
             ('GE', 0.0),
             ('AMD', 0.0),
             ('WMT', 0.0),
             ('BAC', 0.0),
             ('GM', 0.0),
             ('T', 0.6703436046652544),
             ('UAA', 0.0),
             ('SHLD', 0.0),
             ('XOM', 0.0214274069125859),
             ('RRC', 0.0),
             ('BBY', 0.0),
             ('MA', 0.0),
             ('PFE', 0.0),
             ('JPM', 0.0),
             ('SBUX', 0.0)])

In [ ]:
from forex_python.converter import CurrencyRates
import pandas as pd

# Initialize the CurrencyRates object
c = CurrencyRates()

# Example dataframe with dates and prices in EUR
df = pd.DataFrame({
    'date': ['2023-01-01', '2023-01-02'],
    'price_eur': [100, 105]
})

# Convert prices to USD
df['price_usd'] = df.apply(lambda row: row['price_eur'] * c.get_rate('EUR', 'USD', row['date']), axis=1)
print(df)



- Frontline (FRO.OL): NOK (listed on the Oslo Stock Exchange)
- Nordea Bank Abp (NDA-FI.HE): EUR (listed on the Helsinki Stock Exchange)
- BAWAG Group AG (BG.VI): EUR (listed on the Vienna Stock Exchange)
- CaixaBank (CABK.MC): EUR (listed on the Madrid Stock Exchange)
- Danske Bank (DANSKE.CO): DKK (listed on the Copenhagen Stock Exchange)
- McDonald’s (MCD): USD (listed on the NYSE)



In [18]:
import yfinance as yf
import pandas as pd

# List of ticker symbols
tickers = ['FRO.OL', 'NDA-FI.HE', 'BG.VI', 'CABK.MC', 'DANSKE.CO', 'MCD']

# Download historical data for the specified tickers
data = yf.download(tickers, start='2008-01-01', end='2024-09-10')

# Display the first few rows of the data
print(data.head())



[*********************100%***********************]  6 of 6 completed


Price                     Adj Close                                     \
Ticker                        BG.VI   CABK.MC   DANSKE.CO       FRO.OL   
Date                                                                     
2008-01-02 00:00:00+00:00       NaN  2.082304  122.023247  1177.066528   
2008-01-03 00:00:00+00:00       NaN  2.042260  120.215500  1183.767700   
2008-01-04 00:00:00+00:00       NaN  2.042260  117.955788  1125.696045   
2008-01-07 00:00:00+00:00       NaN  1.982193  119.010323  1063.157471   
2008-01-08 00:00:00+00:00       NaN  2.002215  120.215500  1078.791992   

Price                                          Close                      \
Ticker                           MCD NDA-FI.HE BG.VI CABK.MC   DANSKE.CO   
Date                                                                       
2008-01-02 00:00:00+00:00  36.280491  4.380477   NaN    5.20  187.387222   
2008-01-03 00:00:00+00:00  36.174335  4.392222   NaN    5.10  184.611115   
2008-01-04 00:00:00+00:00  

In [2]:
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 35.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 59.7 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.6-py3-none-any.whl size=138891 sha256=7268ab58d7e33080c6514d98a0a74081eedae6ec00d12d55f1d521a4e699374c
  Stored in directory: /home/codespace/.cache/pip/wheels/a6/5e/0f/8319805c4115320e0d3e8fb5799b114a2e4c4a3d6c7e523b06
Successfully built peewee
